In [15]:
from model.dual_ar import DualARTransformer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DualARTransformer.from_pretrained(
    "../checkpoints/smoltts_1227_49000", load_weights=True
)
model = model.to(device)
model = model.to(torch.bfloat16)


Loading model from ../checkpoints/smoltts_1227_49000, config: DualARModelArgs(model_type='dual_ar', vocab_size=51200, n_layer=30, n_head=9, dim=576, intermediate_size=1536, n_local_heads=3, head_dim=64, rope_base=100000, norm_eps=1e-05, max_seq_len=8192, dropout=0.0, tie_word_embeddings=True, attention_qkv_bias=False, codebook_size=2048, num_codebooks=8, use_gradient_checkpointing=True, initializer_range=0.041666666666666664, is_reward_model=False, share_codebook_embeddings=True, scale_codebook_embeddings=False, n_fast_layer=4, fast_dim=576, fast_n_head=9, fast_n_local_heads=3, fast_head_dim=64, fast_intermediate_size=1536, fast_attention_qkv_bias=False)
Loaded weights with error: <All keys matched successfully>


In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../checkpoints/smoltts_1227_49000")

In [21]:
import numpy as np

np.set_printoptions(threshold=np.inf)

tokens = np.load("../train/tokens_batch_1.npy")
first_batch_prompt = tokens[0, :, :20]
first_batch_prompt

array([[    1,  9690,   198, 15024,   494,   578,   260,  2711,  1694,
            2,   198,     1,  4093,   198,    49,  2030,  5610,   436,
         1135,   281],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

In [23]:
tokenizer.decode(first_batch_prompt[0,:])

'<|im_start|>system\nSpeak out the provided text<|im_end|>\n<|im_start|>user\nA further discovery was made in'

In [24]:
res = model.forward(torch.from_numpy(first_batch_prompt).to(device).unsqueeze(0))
res.token_logits.shape

torch.Size([1, 20, 51200])

In [25]:
print(tokenizer.decode(res.token_logits[0,-1,:].argmax()), tokenizer.decode(tokens[0,0,40]))

 in <|semantic:1641|>


In [7]:
res.codebook_logits[0,-1,:].shape

torch.Size([8, 2048])